In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!kaggle datasets download -d ashkhagan/figshare-brain-tumor-dataset

100% 839M/839M [00:37<00:00, 25.0MB/s]
100% 839M/839M [00:37<00:00, 23.4MB/s]


In [3]:
! unzip figshare-brain-tumor-dataset.zip

Archive:  figshare-brain-tumor-dataset.zip
  inflating: dataset/README.txt      
  inflating: dataset/cvind.mat       
  inflating: dataset/data/1.mat      
  inflating: dataset/data/10.mat     
  inflating: dataset/data/100.mat    
  inflating: dataset/data/1000.mat   
  inflating: dataset/data/1001.mat   
  inflating: dataset/data/1002.mat   
  inflating: dataset/data/1003.mat   
  inflating: dataset/data/1004.mat   
  inflating: dataset/data/1005.mat   
  inflating: dataset/data/1006.mat   
  inflating: dataset/data/1007.mat   
  inflating: dataset/data/1008.mat   
  inflating: dataset/data/1009.mat   
  inflating: dataset/data/101.mat    
  inflating: dataset/data/1010.mat   
  inflating: dataset/data/1011.mat   
  inflating: dataset/data/1012.mat   
  inflating: dataset/data/1013.mat   
  inflating: dataset/data/1014.mat   
  inflating: dataset/data/1015.mat   
  inflating: dataset/data/1016.mat   
  inflating: dataset/data/1017.mat   
  inflating: dataset/data/1018.mat   
  infla

In [4]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import h5py
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Input,Concatenate
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
import cv2
from tensorflow import keras

In [5]:
datadir="/content/dataset/data"
N=3064
traindata=[]

for i in range(1,N+1):
    filename=str(i)+".mat"
    data=h5py.File(os.path.join(datadir,filename),"r")
    traindata.append(data)

    if i%100==0:
      print(filename)

100.mat
200.mat
300.mat
400.mat
500.mat
600.mat
700.mat
800.mat
900.mat
1000.mat
1100.mat
1200.mat
1300.mat
1400.mat
1500.mat
1600.mat
1700.mat
1800.mat
1900.mat
2000.mat
2100.mat
2200.mat
2300.mat
2400.mat
2500.mat
2600.mat
2700.mat
2800.mat
2900.mat
3000.mat


In [6]:
import random

random.shuffle(traindata)

In [7]:
train_images = []
train_labels = []
test_images = []
test_labels = []
temp=round(4*3064/5)

for i in range(temp):
  img = traindata[i]["cjdata"]["image"]
  if img.shape==(512,512):
    img=np.expand_dims(img,axis=2)
    train_images.append(img)
    lbl = traindata[i]["cjdata"]["label"]
    label=int(lbl[0])-1
    train_labels.append(label)

for i in range(temp,3064):
  img = traindata[i]["cjdata"]["image"]
  if img.shape==(512,512):
    img=np.expand_dims(img,axis=2)
    test_images.append(img)
    lbl = traindata[i]["cjdata"]["label"]
    label=int(lbl[0])-1
    test_labels.append(label) 

In [8]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [9]:
image_size = [512,512]

In [10]:
img_input = Input(shape=(512,512,1))
img_conc = Concatenate()([img_input, img_input, img_input])  

In [11]:
base_model = InceptionResNetV2(input_tensor = img_conc,weights='imagenet',include_top=False,classes = 1000)

219055592/219055592 [==============================] - 10s 0us/step


In [12]:
base_model.output

<KerasTensor: shape=(None, 14, 14, 1536) dtype=float32 (created by layer 'conv_7b_ac')>

In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [14]:
x = Dense(1024,activation='relu')(x)

In [15]:
preds = Dense(3,activation='softmax')(x)

In [16]:
model = Model(inputs = base_model.input,outputs=preds)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 concatenate (Concatenate)      (None, 512, 512, 3)  0           ['input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 255, 255, 32  864         ['concatenate[0][0]']        

In [17]:
for layer in base_model.layers:
    layer.trainable = False

In [18]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [20]:
r = model.fit(train_images,train_labels,epochs=20,batch_size=32,shuffle = False)

Epoch 1/20
77/77 [==============================] - 66s 862ms/step - loss: 1351.9810 - accuracy: 0.4781
Epoch 2/20
77/77 [==============================] - 65s 848ms/step - loss: 143.7496 - accuracy: 0.4908
Epoch 3/20
77/77 [==============================] - 66s 856ms/step - loss: 96.5490 - accuracy: 0.5383
Epoch 4/20
77/77 [==============================] - 66s 859ms/step - loss: 102.4767 - accuracy: 0.4961
Epoch 5/20
77/77 [==============================] - 67s 864ms/step - loss: 57.9517 - accuracy: 0.5338
Epoch 6/20
77/77 [==============================] - 66s 863ms/step - loss: 38.2445 - accuracy: 0.5457
Epoch 7/20
77/77 [==============================] - 67s 864ms/step - loss: 21.2257 - accuracy: 0.5752
Epoch 8/20
77/77 [==============================] - 67s 864ms/step - loss: 36.2836 - accuracy: 0.5322
Epoch 9/20
77/77 [==============================] - 67s 864ms/step - loss: 22.8042 - accuracy: 0.5609
Epoch 10/20
77/77 [==============================] - 67s 864ms/step - loss: 4.

KeyboardInterrupt: ignored

In [ ]:
result=model.evaluate(test_images,test_labels)
print(result)

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
# plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss_VGGnet')

In [ ]:
label_pred_keras = model.predict(test_images)
print(label_pred_keras)

In [ ]:
label_pred_keras = model.predict(test_images[50][None,...])
print(label_pred_keras)

In [ ]:
final_class = np.unravel_index(label_pred_keras.argmax(),label_pred_keras.shape)
if final_class[1] == 0:
  print("meningioma")
elif final_class[1] == 1:
  print("glioma")
else:
  print("pituitary")

In [ ]:
test_labels[50]

In [21]:
model.save('Inception-ResNet-transfer.h5')

In [23]:
!cp "/content/Inception-ResNet-transfer.h5" "/content/drive/MyDrive/Final_year_project/Pri"